# 问题：647. 回文子串
# Problem: [647. Palindromic Substrings](https://leetcode.com/problems/palindromic-substrings/description/)

## 描述 Description
> ### Given a string, your task is to count how many palindromic substrings in this string.

> ### The substrings with different start indexes or end indexes are counted as different substrings even they consist of same characters.

## 例子 Example

> ![example](https://github.com/Decalogue/AlgorithmMap/blob/master/img/leetcode/647.png "example")

## 分析 Analysis

核心思想：
> 思路1：直接循环求解
>> 时间复杂度：O(n^2)
>> 空间复杂度：O(1)

> 思路2：中心枚举
>> 时间复杂度：O(n^2)
>> 空间复杂度：O(1)

> 思路3：Manacher's Algorithm
>> 时间复杂度：O(n)
>> 空间复杂度：O(n)

> 思路4：动态规划
>> 时间复杂度：O(n^2)
>> 空间复杂度：O(1)

> 思路5：Discuss看到的一个解法
>> 时间复杂度：O(n^2)
>> 空间复杂度：O(1)

## 定义 Definition

### Python

```python
class Solution:
    def countSubstrings(self, s):
```

### C++

```c++
class Solution {
public:
    int countSubstrings(string s) {
        
    }
};
```

### Java

```java
class Solution {
    public int countSubstrings(String s) {
        
    }
}
```

## 解决方案 Solution

### 1.直接循环求解

> 时间复杂度：O(n^2)
> 空间复杂度：O(1)

### Python

### C++

```c++

```

### Java

```java

```

### 2.中心枚举

> 时间复杂度：O(n^2)
> 空间复杂度：O(1)

通过枚举字符串子串的中心而不是起点，向两边同时扩散，依然是逐一判断子串的回文性。这种优化算法比之前第一种算法在最坏的情况下（即只有一种字符的字符串）效率会有很大程度的上升。

### Python

In [5]:
# TODO 待修改为返回回文子串个数
class Solution:
    def longest_palindrome(self, s):
        """
        :type s: str
        :rtype: str
        """
        l = len(s)
        max_length = 0
        palindromic = ''
        radius = []
        for i in range(l):
            x = 1
            while (i - x) >= 0 and (i + x) < l:
                if s[i + x] == s[i - x]:
                    x += 1
                else:
                    break
            x -= 1
            if 2 * x + 1 > max_length:
                max_length = 2 * x + 1
                palindromic = s[i - x:i + x + 1]
            x = 0
            if (i + 1) < l:
                while (i - x) >= 0 and (i + 1 + x) < l:
                    if s[i + 1 + x] == s[i - x]:
                        x += 1
                    else:
                        break
            x -= 1
            if 2 * x + 2 > max_length:
                max_length = 2 * x + 2
                palindromic = s[i - x:i + x + 2]
        if palindromic == '':
            palindromic = s[0]
        return palindromic

### C++

```c++

```

### Java

```Java

```

### 3.Manacher’s Algorithm

> 时间复杂度：O(n)
> 空间复杂度：O(n)

主要步骤:

1.预处理：将所有可能的奇数/偶数长度的回文子串都转换成奇数长度：在 s 每个字符两边都插入一个特殊符号之后变成新 ns。abba => #a#b#b#a#， aba => #a#b#a#。为了进一步减少编码的复杂度，可以在字符串的开始和结尾加入另一个特殊字符，这样就不用特殊处理越界问题，比如 aba => @#a#b#a#$。

2.用数组 radius[i] 来记录以字符 ns[i] 为中心的最长回文子串向左/右扩张的长度（不包括 ns[i]），radius[i] 正好是原字符串中回文串的总长度。增加两个辅助变量 center 和 right，其中 center 为已知的（右边界最大）的回文子串的中心，right 则为 center + radius[center]，也就是这个子串的右边界。

在算法中,center 和 right 是一直更新的，其表示，以 center 为起点，半径为 right 作用范围内能够包含的区间,即[center - right, center + right]。

如果 right > i，那么 radius[i] >= min(radius[2 * center - i], right - i)，其中i与j关于 center 对称。

a.当 right - i > radius[j] 的时候，以 ns[j] 为中心的回文子串包含在以 ns[center] 为中心的回文子串中，由于 i 和 j 对称，以 ns[i] 为中心的回文子串必然包含在以 ns[center] 为中心的回文子串中，所以必有 radius[i] = radius[j] = radius[2 * center - i]，如下图：

> ![manacher_1](https://github.com/Decalogue/AlgorithmMap/blob/master/img/leetcode/647_manacher_1.png "manacher")

当 radius[j] >= right - i 的时候，以 ns[j] 为中心的回文子串不一定完全包含于以 ns[center] 为中心的回文子串中，但是基于对称性可知，下图中两个绿框所包围的部分是相同的，也就是说以 ns[i] 为中心的回文子串，其向右至少会扩张到 right 的位置，也就是说 radius[i] >= right - i。至于 right 之后的部分是否对称，就只能继续去匹配了。

> ![manacher_2](https://github.com/Decalogue/AlgorithmMap/blob/master/img/leetcode/647_manacher_2.png "manacher")

b.如果 right <= i，正在处理的字符不在以 center 为起点的作用区间范围内，则无法用到上一次处理的结果，此时只能去匹配。

### Python

In [1]:
class Solution:
    def countSubstrings(self, s):
        def manachers(s):
            # 字符串处理，用特殊字符隔离字符串，方便处理偶数子串
            ns = '@#' + '#'.join(s) + '#$'
            # radius[i] 来记录以字符 ns[i] 为中心的最长回文子串向左/右扩张的长度（不包括 ns[i]）
            radius = [0] * len(ns)
            center = right = 0
            # maxd = 0 # 记录最长的回文子串长度
            for i in range(1, len(ns) - 1):
                # 字符i在 center 的作用范围内，可以利用之前的结果
                if right > i:
                    radius[i] = min(right - i, radius[2 * center - i])
                # 字符i不在 center 的作用范围内，继续匹配
                while ns[i + radius[i] + 1] == ns[i - radius[i] - 1]:
                    radius[i] += 1
                # 更新 center 及其区间
                if i + radius[i] > right:
                    center, right = i, i + radius[i]
                # maxd = max(maxd, radius[i])
            # return maxd
        # return manachers(s) # 最长的回文子串长度
            return radius
        # 长度为 v 的回文串有 int((v+1)/2) 个子串
        return sum(int((v+1)/2) for v in manachers(s)) # 回文子串的个数（索引不同内容相同子串视为不同子串）

### C++

```c++

```

### Java

```Java

```

### 4.动态规划

> 时间复杂度：O(n^2)
> 空间复杂度：O(1)

### Python

### C++

```c++

```

### Java

```Java

```

### 5.Discuss看到的一个解法

> 时间复杂度：O(n^2)
> 空间复杂度：O(1)

### Python

### C++

```c++

```

### Java

```Java

```

## 总结

### 1.看到这个问题，我最初是怎么思考的？我是怎么做的？遇到了哪些问题？
> 

### 2.别人是怎么思考的？别人是怎么做的？遇到了哪些问题？
> 

### 3.与他的做法相比，我有哪些可以提升的地方？
> 

### 4.整合升级，分享
> 

### 5.迭代第1步到第4步
> 